In [8]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
MODEL_NAME = os.getenv("MODEL_NAME") or ""
BASE_URL = os.getenv("BASE_URL") or ""
API_KEY = os.getenv("API_KEY") or "" 
EMBEDDING_MODEL = os.getenv("EMBEDDING_MODEL") or ""

# LLM

In [12]:
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, SecretStr
from langchain.agents import create_agent
# from components import prompt_with_context

llm = ChatOpenAI(model=MODEL_NAME, base_url=BASE_URL, api_key=SecretStr(API_KEY))
# tools = [retrieve_context]
prompt = (
    "You have access to a tool that retrieves context from a blog post. "
    "Use the tool to help answer user queries."
)
agent = create_agent(
    model=llm, tools=[], system_prompt=prompt, middleware=[]
)

for step in agent.stream(
    {"messages":"hi"},
    stream_mode="values"
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

hi
================================== Ai Message ==================================

Hello! How can I help you today?


In [ ]:
from langchain_ollama import OllamaEmbeddings
embeddings = OllamaEmbeddings(model=EMBEDDING_MODEL)

In [21]:
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

website (blog site)

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

# only keep post title, headers, and content form the full HTML
bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))
loader = WebBaseLoader(
    # web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    web_paths=("https://docs.langchain.com/oss/python/integrations/chat/ollama",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

In [48]:
print(docs[-1].metadata)

{'source': 'https://docs.langchain.com/oss/python/integrations/chat/ollama', 'title': 'ChatOllama - Docs by LangChain', 'description': 'Get started using Ollama chat models in LangChain.', 'language': 'en'}


In [ ]:
print(docs[-1].page_content)

text splitter

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

# print(f"Split blog post into {len(all_splits)} sub-documents.")


document_ids = vector_store.add_documents(documents=all_splits)
print(document_ids[:3])

In [ ]:
from langchain.agents.middleware import dynamic_prompt, ModelRequest


@dynamic_prompt
def prompt_with_context(request: ModelRequest) -> str:
    """Inject context into state messages."""
    last_query = request.state["messages"][-1].text
    retrieved_docs = vector_store.similarity_search(last_query)

    docs_content = "\n\n".join(doc.page_content for doc in retrieved_docs)
    # print(docs_content)
    system_message = (
        "You are a helpful assistant. Use the following context in your response:"
        f"\n\n{docs_content}"
    )

    return system_message

pdf analysis

In [ ]:
from langchain_pymupdf4llm import PyMuPDF4LLMLoader, py

loader = PyMuPDF4LLMLoader(
    file_path="C:/Users/pkmpp/Downloads/1.pdf"
)
docs = await loader.aload()

Consider using the pymupdf_layout package for a greatly improved page layout analysis.


In [8]:
print(docs[-1].page_content)

A retail store wants to introduce an automated discount calculation system to improve
customer billing efficiency.
The discount offered to a customer depends on the total purchase amount.


As a junior data analyst, you are asked to write a simple R program to determine the

applicable discount percentage for a customer based on the purchase amount entered.


Write an R program that performs the following tasks:


1. Prompt the user to enter the total purchase amount.


2. Convert the entered value into a numeric data type.


3. Use if–else if–else conditional statements to determine the discount according to

|the rules below:|Col2|
|---|---|
|Purchase Amount (LKR)|Discount|
|Amount ≥ 5000|20%|
|Amount ≥ 3000|15%|
|Amount ≥ 1000|10%|
|Amount < 1000|5%|



4. Display the applicable discount percentage clearly.


Classification: Public-SLIIT



